In [143]:
# !pip install pytorch_lightning

In [144]:
import random, os, pickle, time
import pandas as pd
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from collections import defaultdict


# Set environment variables for reproducibility and safety
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score, accuracy_score

# 1. Configuration & Seeding
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [145]:
name = 'book'

#### TCKG

In [146]:
class TCKG:
    def __init__(self, tckg_csv_path):
        self.adj_list = defaultdict(list)

        print(f"Loading TCKG from {tckg_csv_path}...")
        df_tckg = pd.read_csv(tckg_csv_path, usecols=['head_id', 'relation_id', 'tail_id'])

        offset = int(df_tckg['relation_id'].max())

        data = df_tckg[['head_id', 'relation_id', 'tail_id']].to_numpy() # Using numpy to speedup
        for h, r, t in data:
            h, r, t = int(h), int(r), int(t)

            self.adj_list[h].append((r, t))
            self.adj_list[t].append((r + offset, h)) # Ex: r=5 (watched) -> r_inv=105 (watched_by)

        print(f"TCKG Loaded successfully. Graph construction complete.")

    def get_neighbors(self, node_id):
        return self.adj_list[node_id]

    def get_all_nodes(self):
        return list(self.adj_list.keys())
        rels = self.rel_matrix[node_ids_tensor]
        tails = self.tail_matrix[node_ids_tensor]
        masks = self.mask_matrix[node_ids_tensor]
        return rels, tails, masks

#### TimeAwareRewardFunction

In [147]:
class TimeAwareRewardFunction(nn.Module):
    def __init__(self, user_embs, entity_embs, relation_embs, interaction_cluster_ids, bias_embs=None, temperature= None):
        """
        Args:
            user_embs (nn.Embedding): Embedding của User (e_u)self.max_neighbors = max(len(edges) for edges in self.adj_list.values())
            entity_embs (nn.Embedding): Embedding của Item/Entity (e_v)
            relation_embs (nn.Embedding): Embedding của Relation (dùng để lấy V_U)
            interaction_cluster_ids (list or tensor): Danh sách các Relation ID đại diện cho Time Clusters.
                                                      Ví dụ: [20, 21, 22] ứng với interacted_0, interacted_1...
                                                      Đây chính là tập {V_U^1, ..., V_U^L}
            bias_embs (nn.Embedding, optional): Bias của entity (b_v). Nếu None sẽ tự khởi tạo.
        """
        super(TimeAwareRewardFunction, self).__init__()

        self.user_embs = user_embs
        self.entity_embs = entity_embs
        self.relation_embs = relation_embs

        # Danh sách ID của các cluster tương tác theo thời gian (V_U)
        # Chuyển thành tensor để tính toán song song
        self.register_buffer('cluster_ids', torch.tensor(interaction_cluster_ids, dtype=torch.long))

        # Entity Bias (b_v) - Eq (11)
        if bias_embs is None:
            num_entities = entity_embs.num_embeddings - 1
            self.bias_embs = nn.Embedding(num_entities + 1, 1, padding_idx = 0)
            nn.init.zeros_(self.bias_embs.weight) # Khởi tạo bias bằng 0
        else:
            self.bias_embs = bias_embs

        if temperature is None:
            self.temperature = self.entity_embs.embedding_dim ** 0.5
        else:
            self.temperature = temperature

    def calculate_weights(self, history_relation_ids):
        """
        Thực hiện Eq (13): Tính trọng số W_hu dựa trên tần suất tương tác chiều Tiến.
        
        Args:
            history_relation_ids: Tensor (Batch, Max_History_Len) - Các relation ID từ user history (đã pad 0).
            
        Returns:
            weights: Tensor (Batch, L) - Vector trọng số thời gian cho L cụm.
        """
        # 1. Chuẩn bị Tensor để Broadcast (So khớp song song trên GPU)
        # hist_expanded: (Batch, History_Len, 1)
        hist_expanded = history_relation_ids.unsqueeze(-1)
        
        # clusters_expanded: (1, 1, L) - Chứa [21, 22, 23, 24]
        clusters_expanded = self.cluster_ids.view(1, 1, -1)
        
        # 2. So khớp và Đếm (Tử số của Eq 13)
        # matches: (Batch, History_Len, L) -> True/1.0 nếu khớp ID cụm thời gian
        matches = (hist_expanded == clusters_expanded).float()
        
        # counts: (Batch, L) -> Tổng số lần user tương tác vào mỗi cụm
        counts = matches.sum(dim=1)

        # 3. Tính tổng số tương tác thực tế q (Mẫu số của Eq 13)
        # Bằng cách tính tổng của 'counts', ta tự động lờ đi toàn bộ Padding (ID 0) 
        # và các relation ID rác (nếu có), vì chúng không match với forward_clusters.
        # q: (Batch, 1)
        q = counts.sum(dim=1, keepdim=True)

        # 4. Chuẩn hóa để ra trọng số (Thêm 1e-9 để tránh lỗi chia cho 0 nếu user chưa có lịch sử)
        # weights: (Batch, L) -> Đảm bảo tổng các phần tử trên mỗi hàng luôn = 1.0 (hoặc 0)
        weights = counts / (q + 1e-9)

        return weights

    def forward(self, user_ids, item_ids, history_relation_ids):
        """
        Tính Reward Score g_R(v | u)

        Args:
            user_ids: (Batch,)
            item_ids: (Batch,) - Item đích (v_hat) mà Agent dự đoán/dừng lại
            history_relation_ids: (Batch, History_Len) - Lịch sử relation của user

        Returns:
            scores: (Batch,) - Điểm reward
        """
        # --- BƯỚC 1: Lấy Embeddings cơ bản ---
        u_e = self.user_embs(user_ids)       # (B, Dim) -> e_u
        
        v_e = self.entity_embs(item_ids)     # (B, Dim) -> e_v
        v_b = self.bias_embs(item_ids).squeeze(-1) # (B,) -> b_v

        # --- BƯỚC 2: Tính Personalized Interaction Relation (Eq 12) ---
        # r_vu^T = W_hu * V_U

        # a. Tính weights (B, L)
        weights = self.calculate_weights(history_relation_ids)

        # b. Lấy embedding của các cluster V_U^1...L
        # cluster_embs shape: (L, Dim)
        cluster_embs = self.relation_embs(self.cluster_ids)

        # c. Tính tổng có trọng số
        # (B, L) x (L, Dim) -> (B, Dim)
        r_interaction = torch.matmul(weights, cluster_embs)

        # --- BƯỚC 3: Tính Score (Eq 11 & Final Eq) ---
        # g = (e_u + r_interaction) . e_v + b_v

        # Dot product: (e_u + r) * e_v
        query_vector = u_e + r_interaction # (B, Dim)
        dot_product = torch.sum(query_vector * v_e, dim=1) # (B,)

        scores = dot_product + v_b # Cộng bias

        # 1. Scale Score (Chia cho nhiệt độ)
        scaled_score = scores / self.temperature

        # 2. Áp dụng Sigmoid
        rewards = torch.sigmoid(scaled_score)

        return rewards


#### TPRecEnvironment

In [148]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

class TPRecEnvironment(nn.Module):
    def __init__(self, tckg, entity_embeddings, relation_embeddings, reward_function, max_path_len=3, history_len=3):
        """
        Môi trường TPRec được thiết kế chuẩn xác theo kiến trúc Sequence State
        """
        super(TPRecEnvironment, self).__init__()
        self.tckg = tckg
        self.entity_embs = entity_embeddings
        self.relation_embs = relation_embeddings
        self.max_path_len = max_path_len
        self.history_len = history_len

        # LƯU REWARD FUNCTION ĐƯỢC TRUYỀN VÀO
        self.reward_function = reward_function

        # State tracking
        self.current_entities = None
        self.current_users = None
        self.path_history = None
        self.step_counter = 0

        # Biến lưu trữ đáp án mục tiêu để tính Terminal Bonus
        self.target_items = None

    def reset(self, user_ids, target_items=None):
        """
        Khởi tạo trạng thái s_0 = (u, u, ∅)
        Cập nhật: Nạp thêm target_items để Trọng tài (Môi trường) cầm sẵn đáp án
        """
        batch_size = user_ids.size(0)

        self.current_users = user_ids
        self.current_entities = user_ids

        # NẠP ĐÁP ÁN: Lưu lại Target Items cho bước tính Reward cuối cùng
        self.target_items = target_items

        # History h_k: store (entity, relation) theo chuẩn paper
        self.path_history = torch.zeros((batch_size, self.history_len * 2),
                                        dtype=torch.long,
                                        device=user_ids.device)

        self.step_counter = 0
        return self._get_state_embedding()

    def _get_state_embedding(self):
        """
        Kết hợp u, h_k, e_k thành một chuỗi duy nhất đưa vào BLSTM
        """
        # 1. Thêm chiều thời gian (unsqueeze) cho u và e_k
        u_emb = self.entity_embs(self.current_users).unsqueeze(1)    # Shape: (B, 1, d)
        e_emb = self.entity_embs(self.current_entities).unsqueeze(1) # Shape: (B, 1, d)

        # 2. Lấy lịch sử e và r
        e_indices = self.path_history[:, 0::2]
        r_indices = self.path_history[:, 1::2]

        e_vecs = self.entity_embs(e_indices)   # Shape: (B, L, d)
        r_vecs = self.relation_embs(r_indices) # Shape: (B, L, d)

        # 3. Trộn xen kẽ e và r thành chuỗi lịch sử [e1, r1, e2, r2...]
        B, L, d = e_vecs.shape
        history_seq = torch.zeros((B, L * 2, d), device=e_vecs.device)
        history_seq[:, 0::2, :] = e_vecs
        history_seq[:, 1::2, :] = r_vecs

        # 4. KẾT NỐI TOÀN BỘ THEO ĐÚNG CÔNG THỨC S_K TRONG PAPER
        full_state_seq = torch.cat([u_emb, history_seq, e_emb], dim=1) # Shape: (B, Seq_Len, d)

        # Trả về ĐÚNG 1 BIẾN DUY NHẤT đại diện cho S_k
        return full_state_seq

    def get_pruned_actions(self, epsilon=15):
        """
        Phiên bản Tối ưu hóa Vectorization (GPU Accelerated)
        """
        batch_size = self.current_users.size(0)
        device = self.current_users.device

        # 1. KÉO VỀ CPU MỘT LẦN DUY NHẤT: Tránh gọi .item() N lần
        curr_nodes_cpu = self.current_entities.tolist()

        # Tra cứu láng giềng cực nhanh trên CPU bằng List Comprehension
        batch_neighbors = [self.tckg.get_neighbors(node) for node in curr_nodes_cpu]

        # =====================================================================
        # 🛑 NGẮT LUÔN LIÊN KẾT: CHỐNG RÒ RỈ DỮ LIỆU (TARGET LEAKAGE)
        # =====================================================================
        if self.target_items is not None:
            targets_cpu = self.target_items.tolist()
            users_cpu = self.current_users.tolist()
            filtered_batch_neighbors = []

            for i, neighbors in enumerate(batch_neighbors):
                # NẾU Agent đang đứng tại đúng User gốc của nó
                if curr_nodes_cpu[i] == users_cpu[i]:
                    target_node = targets_cpu[i]
                    # CHÉM BỎ ĐƯỜNG TẮT: Xóa Target Item khỏi danh sách láng giềng
                    valid_neighbors = [n for n in neighbors if n[1] != target_node]
                    filtered_batch_neighbors.append(valid_neighbors)
                else:
                    # Nếu Agent đang ở Node khác (Item, Tác giả...), cho phép đi bình thường
                    filtered_batch_neighbors.append(neighbors)

            batch_neighbors = filtered_batch_neighbors
        # =====================================================================

        # Tìm node có số lượng láng giềng lớn nhất trong batch này
        lengths = [len(n) for n in batch_neighbors]
        max_len = max(lengths) if lengths else 0

        valid_actions = []

        # Nếu toàn bộ batch đều rơi vào ngõ cụt
        if max_len == 0:
            return [[] for _ in range(batch_size)]

        # 2. TẠO MA TRẬN PADDING TRÊN CPU
        batch_rels = torch.zeros((batch_size, max_len), dtype=torch.long)
        batch_next_nodes = torch.zeros((batch_size, max_len), dtype=torch.long)
        mask = torch.zeros((batch_size, max_len), dtype=torch.bool)

        # Điền dữ liệu vào ma trận
        for i, neighbors in enumerate(batch_neighbors):
            num_n = lengths[i]
            if num_n > 0:
                batch_rels[i, :num_n] = torch.tensor([n[0] for n in neighbors])
                batch_next_nodes[i, :num_n] = torch.tensor([n[1] for n in neighbors])
                mask[i, :num_n] = True

        # 3. ĐẨY TOÀN BỘ MA TRẬN LÊN GPU MỘT LẦN DUY NHẤT
        batch_rels = batch_rels.to(device)
        batch_next_nodes = batch_next_nodes.to(device)
        mask = mask.to(device)

        # 4. TÍNH TOÁN SONG SONG BẰNG MA TRẬN TRÊN GPU
        curr_emb = self.entity_embs(self.current_entities) # Lấy node HIỆN TẠI (Đã sửa)
        r_emb = self.relation_embs(batch_rels)
        n_emb = self.entity_embs(batch_next_nodes)

        # Tính Query: Cần unsqueeze curr_emb để cộng broadcast với r_emb
        query = curr_emb.unsqueeze(1) + r_emb

        # LỖI CŨ CẦN XÓA: scores = torch.sum(query * n_emb, dim=-1)

        # CÁCH CHUẨN MỚI DÀNH CHO TRANSE:
        # Tính khoảng cách L2 (Euclidean distance) giữa (h+r) và t
        # Khoảng cách càng nhỏ càng tốt -> Thêm dấu trừ (-) để hàm topk chọn giá trị gần 0 nhất
        distances = torch.norm(query - n_emb, p=2, dim=-1)
        scores = -distances

        # CHE PADDING (Masking)
        scores = scores.masked_fill(~mask, float('-inf'))

        # 5. CHỌN TOP-K CHO CẢ BATCH
        k = min(epsilon, max_len)
        top_scores, top_indices = torch.topk(scores, k, dim=1) # (B, k)

        # 6. KÉO KẾT QUẢ VỀ LẠI CPU
        top_indices_cpu = top_indices.tolist()
        batch_rels_cpu = batch_rels.tolist()
        batch_nodes_cpu = batch_next_nodes.tolist()

        # Giải nén thành dạng List gốc
        for i in range(batch_size):
            actions_i = []
            num_real = lengths[i]

            if num_real > 0:
                valid_k = min(k, num_real)
                for j in range(valid_k):
                    idx = top_indices_cpu[i][j]
                    actions_i.append((batch_rels_cpu[i][idx], batch_nodes_cpu[i][idx]))

            valid_actions.append(actions_i)

        return valid_actions

    def get_action_space_batch(self):
        """
        Lấy không gian hành động cho cả Batch và Padding thành Tensor.
        """
        raw_actions_list = self.get_pruned_actions()
        batch_size = len(raw_actions_list)

        lengths = [len(acts) for acts in raw_actions_list]
        max_len = max(lengths) if lengths else 0
        if max_len == 0:
            max_len = 1

        device = self.current_entities.device

        r_indices = torch.zeros((batch_size, max_len), dtype=torch.long, device=device)
        e_indices = torch.zeros((batch_size, max_len), dtype=torch.long, device=device)
        action_mask = torch.zeros((batch_size, max_len), dtype=torch.float, device=device)

        for i, actions in enumerate(raw_actions_list):
            num_acts = len(actions)
            if num_acts > 0:
                rs = [a[0] for a in actions]
                es = [a[1] for a in actions]

                r_indices[i, :num_acts] = torch.tensor(rs, device=device)
                e_indices[i, :num_acts] = torch.tensor(es, device=device)
                action_mask[i, :num_acts] = 1.0

        r_emb = self.relation_embs(r_indices)
        e_emb = self.entity_embs(e_indices)

        action_embs = torch.cat([r_emb, e_emb], dim=-1)

        return action_embs, action_mask, raw_actions_list

    def step(self, actions):
        """
        Transition function (Eq 9): Chuyển trạng thái sang bước k+1
        """
        device = self.current_entities.device
        batch_size = len(actions)

        rels_list = [a[0] for a in actions]
        ents_list = [a[1] for a in actions]

        next_relations = torch.tensor(rels_list, dtype=torch.long, device=device)
        next_entities = torch.tensor(ents_list, dtype=torch.long, device=device)

        # Lấy Node hiện tại làm điểm xuất phát (e_{k-1})
        curr_e = self.current_entities.unsqueeze(1)
        new_r = next_relations.unsqueeze(1)

        # Nối lại theo chuẩn Paper: [e_{k-1}, r_k]
        new_entry = torch.cat([curr_e, new_r], dim=1)

        history_shifted = self.path_history[:, 2:]
        self.path_history = torch.cat([history_shifted, new_entry], dim=1)

        self.current_entities = next_entities #tensor([8673, 8614])
        self.step_counter += 1
        done = (self.step_counter >= self.max_path_len)

        return self._get_state_embedding(), done

    def step_with_indices(self, action_indices, raw_actions_list):
        """
        Thực hiện bước đi dựa trên Index (0, 1, 2...) mà Agent chọn.
        """
        selected_real_actions = []
        batch_size = len(action_indices)

        for i in range(batch_size):
            idx = action_indices[i].item()
            user_acts = raw_actions_list[i]

            if len(user_acts) > 0:
                idx = min(idx, len(user_acts) - 1)
                real_action = user_acts[idx]
            else:
                curr_node = self.current_entities[i].item()
                real_action = (0, curr_node) # Dead-end: Tự trỏ về chính nó

            selected_real_actions.append(real_action)

        return self.step(selected_real_actions)

    def get_reward(self):
        """
        Gọi khi done=True. Tính Time-aware Reward g_R(v|u) + Terminal Bonus
        """
        user_ids = self.current_users
        item_ids = self.current_entities

        batch_history_relations = [] # Chứa danh sách các tensor của từng user
        
        for user_id in user_ids.tolist():   # .tolist() to transfer from gpu to cpu
            batch_neighbors = self.tckg.get_neighbors(user_id)            
            
            # 1. Lấy toàn bộ relation của user HIỆN TẠI
            user_rels = [rel for (rel, tail) in batch_neighbors]
            
            # 2. Chuyển list của user này thành tensor 1D và đưa vào danh sách batch
            user_rels_tensor = torch.tensor(user_rels, dtype=torch.long)
            batch_history_relations.append(user_rels_tensor)
        
        # 3. Padding và gộp thành batch (Tensor 2D)
        # batch_first=True để output có dạng [batch_size, max_seq_len]
        # padding_value=0 (Thay số 0 bằng ID padding thực tế của bạn nếu cần)
        history_relation_ids = pad_sequence(
            batch_history_relations, 
            batch_first=True, 
            padding_value=0
        )
        
        # 4. (Tùy chọn) Đưa tensor trả về đúng thiết bị (GPU) của mô hình
        history_relation_ids = history_relation_ids.to(self.current_users.device)

        # 1. SOFT REWARD (Điểm dẫn đường)
        rewards = self.reward_function(user_ids, item_ids, history_relation_ids)

        return rewards

#### TPRecPolicy

In [149]:
import torch
import torch.nn as nn

class TPRecPolicy(nn.Module):
    def __init__(self, embed_dim, hidden_dim=128, dropout=0.1):
        super(TPRecPolicy, self).__init__()

        # BLSTM giờ đây chỉ cần nhận input_size = embed_dim (không cần nhân 2 nữa)
        self.blstm = nn.LSTM(input_size=embed_dim,
                             hidden_size=hidden_dim // 2,
                             num_layers=1,
                             batch_first=True,
                             bidirectional=True)

        # W1 giờ đây chỉ cần nhận đầu ra của BLSTM
        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

        self.Wa = nn.Linear(hidden_dim, embed_dim * 2)
        self.Wc = nn.Linear(hidden_dim, 1)

    def forward(self, full_state_seq, action_embs, action_mask):
        # 1. Đưa toàn bộ s_k vào BLSTM như công thức (15)
        lstm_out, _ = self.blstm(full_state_seq)

        # # Lấy trạng thái ở bước thời gian cuối cùng đại diện cho toàn bộ chuỗi
        # lstm_last = lstm_out[:, -1, :]

        # Vì bidirectional=True, hidden_dim này thực chất là 2 phần ghép lại
        half_dim = self.blstm.hidden_size

        # Lấy trạng thái Tóm tắt Chiều đi tới (Ở index cuối cùng: -1)
        forward_last = lstm_out[:, -1, :half_dim]

        # Lấy trạng thái Tóm tắt Chiều đi lùi (Ở index đầu tiên: 0)
        backward_last = lstm_out[:, 0, half_dim:]

        # Ghép 2 tóm tắt này lại thành một Context Vector hoàn hảo
        lstm_last = torch.cat([forward_last, backward_last], dim=-1)

        # 2. Tính x_k theo đúng phương trình
        x_k = self.dropout(torch.relu(self.W1(lstm_last)))

        # --- (Phần tính Actor và Critic giữ nguyên) ---
        query = self.Wa(x_k).unsqueeze(1)
        scores = torch.sum(query * action_embs, dim=-1)
        scores = scores.masked_fill(action_mask.bool() == False, float('-inf'))
        probs = torch.softmax(scores, dim=-1)

        value_baseline = self.Wc(x_k).squeeze(-1)

        return probs, value_baseline

#### TPRecLightningModel

In [ ]:
import pytorch_lightning as pl
import torch
import torch.optim as optim

class TPRecLightningModel(pl.LightningModule):
    def __init__(self, env, policy_net, learning_rate=1e-3, beta_entropy=0.01):
        super().__init__()

        # Lưu lại hyperparameter (tự động log vào tensorboard nếu dùng)
        self.save_hyperparameters(ignore=['env', 'policy_net'])

        self.env = env
        self.policy_net = policy_net
        self.learning_rate = learning_rate
        self.beta_entropy = beta_entropy

    def forward(self, full_state_seq, action_embs, action_mask):
        """
        Đạo diễn Lightning chuyển tiếp ĐÚNG 3 tham số cho Diễn viên Policy Network
        """
        return self.policy_net(full_state_seq, action_embs, action_mask)

    def training_step(self, batch, batch_idx):
        """
        Nơi diễn ra toàn bộ chuyến đi của Agent trong 1 Batch
        Bổ sung: Tính toán HR@10 và NDCG@10 trên tập Train
        """
        # 1. Hứng cả User và Target Item từ DataLoader
        if isinstance(batch, (list, tuple)) and len(batch) >= 2:
            batch_users, target_items = batch[0], batch[1]
        else:
            batch_users = batch[0] if isinstance(batch, (list, tuple)) else batch
            target_items = None

        batch_size = batch_users.size(0)

        # 2. Gọi reset và truyền cả target_items để Env tính Bonus cuối cùng
        full_state_seq = self.env.reset(batch_users, target_items)

        saved_log_probs = []
        saved_values = []
        saved_entropies = []

        for t in range(self.env.max_path_len):
            action_embs, action_mask, raw_actions = self.env.get_action_space_batch()

            # 3. Truyền 1 biến state duy nhất vào Policy
            probs, value_baseline = self(full_state_seq, action_embs, action_mask)

            # --- [ĐOẠN MỚI THÊM] TÍNH TRAIN METRICS Ở BƯỚC CUỐI CÙNG ---
            if t == self.env.max_path_len - 1 and target_items is not None:
                k = min(10, probs.size(1))
                _, topk_indices = torch.topk(probs, k=k, dim=1) # (Batch, k)

                top10_list = []
                for i in range(batch_size):
                    user_acts = raw_actions[i]
                    items_i = []
                    for idx in topk_indices[i].tolist():
                        if idx < len(user_acts):
                            items_i.append(user_acts[idx][1])
                    while len(items_i) < 10:
                        items_i.append(0)
                    top10_list.append(items_i)

                final_items_top10 = torch.tensor(top10_list, device=self.device)
                target_expanded = target_items.unsqueeze(1)

                # Chấm điểm Hit Ratio
                hits_matrix = (final_items_top10 == target_expanded).float()
                hr_at_10 = hits_matrix.sum(dim=1).clamp(max=1.0).mean()

                # Chấm điểm NDCG
                ranks = torch.arange(1, 11, device=self.device).float()
                discount = 1.0 / torch.log2(ranks + 1)
                ndcg_at_10 = (hits_matrix * discount).sum(dim=1).mean()

                # Ghi nhận vào hệ thống Log
                self.log('train_hr@10', hr_at_10, prog_bar=True, on_step=False, on_epoch=True)
                self.log('train_ndcg@10', ndcg_at_10, prog_bar=True, on_step=False, on_epoch=True)
            # -----------------------------------------------------------

            # 4. Agent vẫn lấy mẫu ngẫu nhiên (sample) để phục vụ cho RL
            m = torch.distributions.Categorical(probs)
            action_indices = m.sample()

            saved_log_probs.append(m.log_prob(action_indices))
            saved_values.append(value_baseline)
            saved_entropies.append(m.entropy())

            # 5. Hứng State mới và bước sang chặng tiếp theo
            full_state_seq, done = self.env.step_with_indices(action_indices, raw_actions)

        # =====================================================================
        # TÍNH TOÁN LOSS THEO CÔNG THỨC (17) VÀ (18) CỦA BÀI BÁO
        # =====================================================================
        final_rewards = self.env.M().detach() # Shape: (Batch,)
        
        # 1. CÔNG THỨC 17: Tính Expected Reward (G) có chiết khấu \gamma
        # \sum_{t=0}^{K-1} \gamma^t R_{k+1}
        gamma = 0.99 # Hệ số chiết khấu chuẩn trong RL
        returns = []
        R = final_rewards # Phần thưởng nhận được ở bước cuối cùng
        
        # Tính ngược từ bước cuối về bước đầu
        for step in reversed(range(self.env.max_path_len)):
            returns.insert(0, R)
            R = R * gamma # Chiết khấu lùi về quá khứ 1 bước

        policy_loss = 0
        value_loss = 0

        # 2. CÔNG THỨC 18: Tối ưu REINFORCE Algorithm
        # \nabla_\Theta \log \pi_\Theta (G - \hat{c}(s_k))
        for G, log_prob, value_baseline in zip(returns, saved_log_probs, saved_values):
            
            # (G - \hat{c}(s_k)): Tính Advantage (Sự chênh lệch giữa Thực tế G và Kỳ vọng baseline)
            advantage = G - value_baseline.detach()
            
            # \nabla_\Theta \log \pi_\Theta * Advantage
            # Thêm dấu TRỪ vì PyTorch tự động Minimize Loss, trong khi ta muốn Maximize Reward
            step_policy_loss = -log_prob * advantage
            policy_loss += step_policy_loss.mean() # Lấy trung bình cho cả Batch
            
            # Huấn luyện mạng Critic (Wa) để dự đoán G chuẩn xác hơn bằng MSE
            step_value_loss = torch.nn.functional.mse_loss(value_baseline, G)
            value_loss += step_value_loss

        # Tổng hợp Loss cho toàn bộ đường đi
        # (Bạn có thể trừ đi entropy_bonus ở đây nếu muốn Agent khám phá nhiều hơn)
        total_loss = policy_loss + value_loss 

        # Ghi nhận log
        self.log('train_reward', final_rewards.mean(), prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_loss', total_loss, prog_bar=False, on_step=False, on_epoch=True)

        return total_loss

    def validation_step(self, batch, batch_idx):
        """
        Đánh giá mô hình: Tính HR@10 và NDCG@10 tại bước cuối cùng.
        """
        # 1. Hứng biến từ Dataloader
        if isinstance(batch, (list, tuple)) and len(batch) >= 2:
            batch_users, target_items = batch[0], batch[1]
        else:
            batch_users = batch[0] if isinstance(batch, (list, tuple)) else batch
            target_items = None

        # 2. Reset Env (Truyền target_items để tính reward giống Train)
        full_state_seq = self.env.reset(batch_users, target_items)
        batch_size = batch_users.size(0)

        final_items_top10 = None

        # BẮT ĐẦU ĐI TÌM ĐƯỜNG
        for t in range(self.env.max_path_len):
            action_embs, action_mask, raw_actions = self.env.get_action_space_batch()

            # 3. Truyền 1 biến state duy nhất vào Policy
            probs, _ = self(full_state_seq, action_embs, action_mask)

            # NẾU LÀ BƯỚC CUỐI CÙNG: Lấy Top 10 thay vì Top 1
            if t == self.env.max_path_len - 1:
                k = min(10, probs.size(1))
                _, topk_indices = torch.topk(probs, k=k, dim=1) # (Batch, k)

                top10_list = []
                for i in range(batch_size):
                    user_acts = raw_actions[i]
                    items_i = []

                    for idx in topk_indices[i].tolist():
                        if idx < len(user_acts):
                            items_i.append(user_acts[idx][1])

                    while len(items_i) < 10:
                        items_i.append(0)

                    top10_list.append(items_i)

                final_items_top10 = torch.tensor(top10_list, device=self.device)
                action_indices = topk_indices[:, 0]
            else:
                action_indices = torch.argmax(probs, dim=1)

            # 4. Hứng đúng 2 biến giống hệt bên Train
            full_state_seq, done = self.env.step_with_indices(action_indices, raw_actions)

        # Tính toán và Log phần thưởng (Reward)
        val_rewards = self.env.get_reward().detach()
        self.log('val_reward', val_rewards.mean(), prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)

        # --- ĐÁNH GIÁ HIT RATIO @10 & NDCG @10 ---
        if target_items is not None and final_items_top10 is not None:
            target_expanded = target_items.unsqueeze(1)
            hits_matrix = (final_items_top10 == target_expanded).float()

            hits_per_user = hits_matrix.sum(dim=1).clamp(max=1.0)
            hr_at_10 = hits_per_user.mean()
            self.log('val_hr@10', hr_at_10, prog_bar=True, on_epoch=True, sync_dist=True)

            ranks = torch.arange(1, 11, device=self.device).float()
            discount = 1.0 / torch.log2(ranks + 1)

            ndcg_per_user = (hits_matrix * discount).sum(dim=1)
            ndcg_at_10 = ndcg_per_user.mean()
            self.log('val_ndcg@10', ndcg_at_10, prog_bar=True, on_epoch=True, sync_dist=True)

    def configure_optimizers(self):
        # Lọc ra CHỈ những tham số đang requires_grad=True (Mạng Policy)
        # Giúp tránh lỗi tối ưu hóa các tham số đã bị freeze (như Embeddings)
        trainable_params = filter(lambda p: p.requires_grad, self.parameters())

        optimizer = optim.Adam(trainable_params, lr=self.learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)

        return [optimizer], [scheduler]

### Class Config

In [151]:
class Config:
    # Đường dẫn dữ liệu
    TCKG_PATH = f"./data/{name}/{name}_TCKG.csv"  # File CSV chứa đồ thị (như bạn đã gửi)
    # TRAIN_USERS_PATH = "data/train_users.csv" # File chứa danh sách user ID dùng để train
    SAVE_DIR = "./checkpoints"

    # Siêu tham số Model
    EMBED_DIM = 64
    HIDDEN_DIM = 128
    HISTORY_LEN = 3   # k' (độ dài lịch sử)
    MAX_PATH_LEN = 3  # K (số bước đi)

    # Siêu tham số Training
    BATCH_SIZE = 512 # Batch lớn giúp RL ổn định hơn
    NUM_EPOCHS = 500
    LEARNING_RATE = 1e-3
    BETA_ENTROPY = 0.01

    # Thiết bị
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # IDs của các Relation Interaction (Quan trọng cho Reward)
    # Ví dụ: 20=interacted_0, 21=interacted_1, 22=interacted_2
    INTERACTION_CLUSTER_IDS = [21, 22, 23, 24]

### Load Dataset

In [152]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class InteractionDataset(Dataset):
    def __init__(self, df):
        """Nhận vào một DataFrame và chuyển đổi các cột cần thiết thành mảng NumPy"""
        self.users = df['user_id'].values
        self.entities = df['entity_id'].values # ĐÂY CHÍNH LÀ TARGET ITEM

    def __len__(self):
        """Trả về tổng số dòng dữ liệu"""
        return len(self.users)

    def __getitem__(self, idx):
        """Lấy dữ liệu tại vị trí idx và biến thành PyTorch Tensor"""
        user_tensor = torch.tensor(self.users[idx], dtype=torch.long)
        entity_tensor = torch.tensor(self.entities[idx], dtype=torch.long)

        # ĐÃ SỬA: Trả về một Tuple gồm (User, Target_Item)
        return user_tensor, entity_tensor

# 2. Đọc file CSV thành Pandas DataFrame
# (Lưu ý nhỏ: Mình giữ nguyên tên file của bạn, nhưng chữ 'interacions' hình như đang thiếu chữ 't', bạn nhớ kiểm tra lại tên file thật nhé)

train_df = pd.read_csv(f'./data/{name}/{name}_train_interactions.csv')
val_df = pd.read_csv(f'./data/{name}/{name}_val_interactions.csv')
test_df = pd.read_csv(f'./data/{name}/{name}_test_interactions.csv')

# 3. Bọc DataFrame vào class Dataset vừa tạo
train_dataset = InteractionDataset(train_df)
val_dataset = InteractionDataset(val_df)
test_dataset = InteractionDataset(test_df)

# 4. Đưa Dataset vào DataLoader
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

### Main function

In [153]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

cfg = Config()
print("🔥 Starting TPRec Training with PyTorch Lightning...")

# -------------------------------------------------
# BƯỚC 1: Xây dựng Đồ thị Tri thức (TCKG)
# -------------------------------------------------
# Lưu ý: Cần đảm bảo file CSV đã được map ID về dạng số nguyên liên tục (0, 1, 2...)
print("Loading Knowledge Graph...")
# Tính offset tự động (như class TCKG tối ưu tôi đã viết)
tckg = TCKG(cfg.TCKG_PATH)

# -------------------------------------------------
# BƯỚC 2: Khởi tạo Embeddings từ file Pickle
# -------------------------------------------------
print("Loading Pre-trained TransE Embeddings...")
pickle_file_path = f'./pickle/{name}_transE_embeddings_2026-02-22_13-17-36.pkl'

with open(pickle_file_path, 'rb') as f:
    saved_data = pickle.load(f)

pretrained_ent = saved_data['entity_embeddings']
pretrained_rel = saved_data['relation_embeddings']

# 2. Chuyển đổi sang PyTorch Tensor (ép kiểu Float32 để tính toán neural network)
# Nếu data đang là Numpy array:
if isinstance(pretrained_ent, np.ndarray):
    ent_tensor = torch.tensor(pretrained_ent, dtype=torch.float32)
    rel_tensor = torch.tensor(pretrained_rel, dtype=torch.float32)
else:
    # Nếu data đã là Tensor sẵn:
    ent_tensor = pretrained_ent.clone().detach().float()
    rel_tensor = pretrained_rel.clone().detach().float()

# 3. Nạp vào nn.Embedding
# freeze=False: Cho phép RL Agent tiếp tục cập nhật (fine-tune) vector trong lúc tìm đường
# freeze=True: Khóa cứng vector, RL Agent chỉ học Policy Network (Học nhanh hơn, chống overfit)
entity_embs = nn.Embedding.from_pretrained(ent_tensor, freeze=True, padding_idx=0)
relation_embs = nn.Embedding.from_pretrained(rel_tensor, freeze=True, padding_idx=0)


# -------------------------------------------------
# BƯỚC 3: Khởi tạo Reward Function
# -------------------------------------------------
print("Setting up Reward Function...")
reward_func = TimeAwareRewardFunction(
    user_embs=entity_embs,    # Chia sẻ trọng số với Env
    entity_embs=entity_embs,
    relation_embs=relation_embs,
    interaction_cluster_ids=cfg.INTERACTION_CLUSTER_IDS,
    bias_embs=None, # Tự tạo bias mới
    temperature= None
)

# -------------------------------------------------
# BƯỚC 4: Khởi tạo Môi trường (Environment)
# -------------------------------------------------
print("Setting up Environment...")
env = TPRecEnvironment(
    tckg=tckg,
    entity_embeddings=entity_embs,
    relation_embeddings=relation_embs,
    reward_function=reward_func, # Inject reward vào env
    max_path_len=cfg.MAX_PATH_LEN,
    history_len=cfg.HISTORY_LEN
)

# -------------------------------------------------
# BƯỚC 5: Khởi tạo Policy Network (Agent)
# -------------------------------------------------
print("Building Policy Network...")

# Tính toán kích thước State và Action theo công thức chuẩn
# Action = Relation + Entity
action_dim = cfg.EMBED_DIM + cfg.EMBED_DIM

# State = User + Flattened_History + Current_Entity
# Flattened_History = k' * (Relation + Entity)
history_flat_dim = cfg.HISTORY_LEN * (cfg.EMBED_DIM + cfg.EMBED_DIM)
state_dim = cfg.EMBED_DIM + history_flat_dim + cfg.EMBED_DIM

print(f"--> State Dim: {state_dim} | Action Dim: {action_dim}")

policy_net = TPRecPolicy(
        embed_dim=cfg.EMBED_DIM,      # Ví dụ: 64
        hidden_dim=cfg.HIDDEN_DIM,    # Ví dụ: 128
        dropout=0.1                   # Tỉ lệ dropout giúp chống Overfit (theo paper)
    )

# BƯỚC 6: ĐÓNG GÓI VÀO LIGHTNING MODEL
print("Packing into Lightning Module...")
lightning_model = TPRecLightningModel(
    env=env,
    policy_net=policy_net,
    learning_rate=cfg.LEARNING_RATE,
    beta_entropy=cfg.BETA_ENTROPY
)

# BƯỚC 7: CẤU HÌNH LƯU BEST MODEL (CHECKPOINT)
# Tự động theo dõi 'train_reward' ở cuối mỗi epoch và lưu lại bản có điểm cao nhất
checkpoint_callback = ModelCheckpoint(
    dirpath=cfg.SAVE_DIR,
    filename='tprec-best-{epoch:02d}-{train_reward:.4f}',
    monitor='train_hr@10',
    mode='max', # Lưu model có reward lớn nhất
    save_top_k=1,
    save_last=True # Lưu thêm model ở epoch cuối cùng để phòng hờ
)

# 2. THÊM CALLBACK EARLY STOPPING VÀO ĐÂY
early_stop_callback = EarlyStopping(
      monitor='val_hr@10',   # Phải cùng tên với biến monitor ở Checkpoint
      min_delta=0.001,       # Sự thay đổi tối thiểu để được tính là "có cải thiện"
      patience=500,           # Sức chịu đựng: Cho phép mô hình dậm chân tại chỗ tối đa 10 Epoch
      verbose=True,          # Bật in thông báo ra màn hình khi Early Stop kích hoạt
      mode='max'             # 'max' vì ta muốn chỉ số HR@10/Reward càng lớn càng tốt
  )

# BƯỚC 8: KHỞI TẠO TRAINER VÀ BẮT ĐẦU CHẠY
print("Initializing Lightning Trainer...")
trainer = pl.Trainer(
    max_epochs=cfg.NUM_EPOCHS,
    accelerator="auto", # Tự động tìm và dùng GPU nếu có
    devices=1,
    gradient_clip_val=1.0, # Tự động áp dụng Gradient Clipping
    callbacks=[checkpoint_callback, early_stop_callback],
    enable_progress_bar=True,
    # log_every_n_steps=10,
    num_sanity_val_steps=0
)

print("🚀 Bắt đầu huấn luyện...")
# DataLoader của bạn cần được truyền vào đây
trainer.fit(
        model=lightning_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )


🔥 Starting TPRec Training with PyTorch Lightning...
Loading Knowledge Graph...
Loading TCKG from ./data/book/book_TCKG.csv...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name       | Type             | Params | Mode  | FLOPs
----------------------------------------------------------------
0 | env        | TPRecEnvironment | 2.5 M  | train | 0    
1 | policy_net | TPRecPolicy      | 99.7 K | train | 0    
----------------------------------------------------------------
138 K     Trainable params
2.5 M     Non-trainable params
2.6 M     Total params
10.522    Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
0         Total Flops


TCKG Loaded successfully. Graph construction complete.
Loading Pre-trained TransE Embeddings...
Setting up Reward Function...
Setting up Environment...
Building Policy Network...
--> State Dim: 512 | Action Dim: 128
Packing into Lightning Module...
Initializing Lightning Trainer...
🚀 Bắt đầu huấn luyện...
Epoch 0:   1%|          | 1/147 [00:00<01:05,  2.22it/s, v_num=40]

3662.67s - thread._ident is None in _get_related_thread!


Epoch 0: 100%|██████████| 147/147 [00:37<00:00,  3.93it/s, v_num=40]

...

Epoch 30:   0%|          | 0/147 [00:00<?, ?it/s, v_num=40, val_reward=0.502, val_hr@10=0.0308, val_ndcg@10=0.0167, train_hr@10=0.033, train_ndcg@10=0.0175, train_reward=0.502]           

4785.48s - thread._ident is None in _get_related_thread!


Epoch 32:  55%|█████▌    | 81/147 [00:21<00:17,  3.78it/s, v_num=40, val_reward=0.502, val_hr@10=0.0308, val_ndcg@10=0.0167, train_hr@10=0.033, train_ndcg@10=0.0173, train_reward=0.502]  


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

In [ ]:
trainer.validate(dataloaders=test_loader, ckpt_path='best')